<a href="https://colab.research.google.com/github/roberttnovak/MedicineScraper/blob/main/playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medicine Scraper

<img src="https://cima.aemps.es/cima/resources/images/CimaTranspaPeq.gif">


In [ ]:
# Install dependencies
!apt update
!apt install chromium-chromedriver
!pip install selenium

In [2]:
# Import modules
import re
from time import sleep
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By #Para poder usar By. dentro de find_element
from selenium.webdriver.support.ui import WebDriverWait #Para esperar hasta cargar la página 
from selenium.webdriver.support import expected_conditions as EC #Para esperar hasta cargar la página 


# Set driver options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # No interface
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--user-agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'")

## **Código base**


In [105]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get('https://cima.aemps.es/cima/publico/home.html')
driver.find_element(By.CSS_SELECTOR,"button[onclick='biosimilaresTodos()']") # Example search in the homepage

# Navigate to "listado de búsqueda"
buscador = driver.find_element(By.ID, "inputbuscadorsimple")
buscador.clear()
buscador.send_keys("*")
buscador.send_keys(Keys.ENTER)
sleep(3)  # If we do not wait, the `page_source` will not have time to change and it will be showed the previous page (the homepage in this case)
driver.title

'.:: CIMA ::. Resultados de la búsqueda de medicamentos'

In [ ]:
# En la página aparecen una serie de filtros por defecto. 
# Nos aseguramos de que no se aplique ningún filtro comprobando, en primer lugar, cuáles son los filtros aplicados por defecto
# y, en segundo lugar, deseleccionandolos. De esta forma, obtenemos todos los medicamentos 

from selenium.common.exceptions import NoSuchElementException
filtros_busqueda = {
    "filtroRecetaSi"              :  "filtro de los medicamentos con receta"                      ,
    "filtroRecetaNo"             :  "filtro de los medicamentes sin receta"                      ,                                    
    "filtroTrianguloSi"          :  "filtro de los medicamentos con seguimiento adicional"       ,
    "filtroTrianguloNo"          :  "filtro de los medicamentos sin seguimiento adicional"       ,
    "filtroHuerfanoSi"           :  "filtro de los medicamentos que son huérfanos"               ,
    "filtroHuerfanoNo"           :  "filtro de los medicamentos que no son huérfanos"            ,
    "filtroBiosimilarSi"         :  "filtro de los medicamentos que son biosimilares"            ,
    "filtroBiosimilarNo"         :  "filtro de los medicamentos que no son biosimilares"         ,
    "filtroComercializadoSi"     :  "filtro de los medicamentos comercializados"                 ,
    "filtroComercializadoNo"     :  "filtro de los medicamentos no comercializados"              ,
    "filtroImpParalelasSi"       :  "filtro de los medicamentos con importación paralela"        ,
    "filtroImpParalelasNo"       :  "filtro de los medicamentos sin importación paralela"        ,
    "filtroAutorizado"           :  "filtro de los medicamentos autorizados"                     ,
    "filtroSuspendido"           :  "filtro de los medicamentos suspendidos"                     ,
    "filtroRevocado"             :  "filtro de los medicamentos revocados"                       ,
    "filtroBiologicos"           :  "filtro de los medicamentos biológicos"                      ,
    "filtroPactivos"             :  "filtro de los medicamentos con estrecho margen terapéutico" ,
    "filtroApRespiratorio"       :  "filtro de los medicamentos por vía respiratoria"                                   
}
for i,v in filtros_busqueda.items():
    try:  #Nos aseguramos de que el código se pueda ejecutar si salta algún error
        if not driver.find_element(By.ID, i).is_selected():
            checkbox=driver.find_element(By.CSS_SELECTOR, "label[for={}]".format(i))
            driver.execute_script("arguments[0].click();", checkbox)
    except NoSuchElementException: #Identificamos el error correspondiente al de los elementos que identifican los checkbox
        print("El {} ya no existe en la página web o su id en el html ha cambiado de nombre".format(v))
    except:
        print("Ha ocurrido un error para el {}".format(v))

In [ ]:
# # References: https://devqa.io/selenium-css-selectors/ 
# # Search a specific medicine
# med_81012 = driver.find_element(By.CSS_SELECTOR, "div[onclick=\"medicamentoOnSelect('81012')\"]")
# med_81012.click()
# print(driver.page_source)

In [ ]:
# Other queries
# driver.find_elements(By.CSS_SELECTOR, "div.list-group-item.row div") # Select all div children inside <div class='list-group-item.row'>
# driver.find_elements(By.CSS_SELECTOR, "#detalleMedicamento > div") # Select all div children inside tag with id=detalleMedicamento

In [ ]:
# Search list of medicines
meds = driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]")
print(len(meds))
meds

25


[<selenium.webdriver.remote.webelement.WebElement (session="fea01d7646566f85501ffad0c4357308", element="f8ff9d4a-1f0d-4e5f-8b28-16db35be774d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fea01d7646566f85501ffad0c4357308", element="410c1fe8-42e4-45bc-a299-8d404b31ab8c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fea01d7646566f85501ffad0c4357308", element="eb0147f2-6688-4d84-a667-caa6a6b11fff")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fea01d7646566f85501ffad0c4357308", element="bf52cd42-c250-4439-bfc0-605e13543169")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fea01d7646566f85501ffad0c4357308", element="e0997b9d-5ef9-4acd-a06f-82485f002dd7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fea01d7646566f85501ffad0c4357308", element="ea3de4e0-8cad-452e-b9e0-65033acda3f7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fea01d7646566f85501ffad0c4357308", element="190c10cd-e67e-4c80-91f1-61

In [ ]:
# Retrieve all ids within the list of medicines

meds_ids = []

for m in meds:
  id = m.get_attribute('onclick')
  meds_ids.append(id)

meds_ids

["medicamentoOnSelect('81012')",
 "medicamentoOnSelect('84182')",
 "medicamentoOnSelect('80803')",
 "medicamentoOnSelect('82913')",
 "medicamentoOnSelect('07428001')",
 "medicamentoOnSelect('65382')",
 "medicamentoOnSelect('65059')",
 "medicamentoOnSelect('61836')",
 "medicamentoOnSelect('61837')",
 "medicamentoOnSelect('60014')",
 "medicamentoOnSelect('70452')",
 "medicamentoOnSelect('70451')",
 "medicamentoOnSelect('69223')",
 "medicamentoOnSelect('71437')",
 "medicamentoOnSelect('79696')",
 "medicamentoOnSelect('62389')",
 "medicamentoOnSelect('66460')",
 "medicamentoOnSelect('66459')",
 "medicamentoOnSelect('67763')",
 "medicamentoOnSelect('66458')",
 "medicamentoOnSelect('67621')",
 "medicamentoOnSelect('67988')",
 "medicamentoOnSelect('67874')",
 "medicamentoOnSelect('64920')",
 "medicamentoOnSelect('64921')"]

In [3]:
def scrape_data(html: str) -> pd.Series:
    #Utilizamos un objeto BeautifulSoup para scrapear la página
    bs = BeautifulSoup(html,"html.parser") 
    
    #A continuación se scrapean todos los elementos presentes en la página:
    
    #Nombre del medicamento: Está entre identificado por la tag h1
    medicamento = bs.find('h1').get_text()
    
    #Nombre del laboratorio: Lo identifica la id="nombrelabXS"
    laboratorio = bs.find('div', {'id':"nombrelabXS"}).get_text()
    
    #Número de registro:  Lo identifica la id="nregistroId"
    num_registro = bs.find('span', {'id': 'nregistroId'}).get_text()
    
    #Comprobamos si un medicamento se ha autorizado o no.
    #Esta información está contenida en la tag h2 identificada por el atributo id=estadoXS
    #Si no está autorizado, entonces la tag esta vacía. Si sí lo está entonces aparece el contenido no vacío con el formato "Autorizado ( dd/mm/aaaa )"
    autorizado_bool = not bs.find('h2', {'id': 'estadoXS'}).get_text() == ''
    if autorizado_bool: 
        autorizado_fecha = re.sub( 
        "\( | \)", #Quitamos los paréntesis y los espacios
        ""
        ,re.findall(
            "\(.*\)", #Encontramos mediante una expresión regular el contenido de la fecha 
            bs.find('h2', {'id': 'estadoXS'}).get_text())[0]
    ).strip() #Nos aseguramos de que no hay ningún espacio adicional 
    else : 
        autorizado_fecha = None
    
    #Explicación análoga al apartado anterior para, en este caso, el estado de suspendido 
    suspendido_bool = not bs.find('h2', {'id': 'estadoXSsec'}).get_text() == ''
    if suspendido_bool: 
        suspendido_fecha = re.sub( 
        "\( | \)",
        ""
        ,re.findall(
            "\(.*\)",
            bs.find('h2', {'id': 'estadoXSsec'}).get_text())[0]
    ).strip()
    else : 
        suspendido_fecha = None
    
    #Si el medicamento está o no comercializado se identifica con un tag h3 identificado por la id='estadocomercXS'
    comercializado_bool = not bs.find('h3', {'id': 'estadocomercXS'}) == None
    
    #Las siguientes columnas son listas que tienen todas la misma estructura: Una etiqueta div con una id que la identifica.  
    #Para extraer la información accedemos a la id correspondiente recorriendo todos los elementos de la lista (del html) y 
    #guardándola en una lista (de python)
    
    vias_administracion = bs.find('div', {'id':'viasadministracion'}).find_all('li')
    vias_administracion = [va.get_text() for va in vias_administracion]
    
    dosis = bs.find('div', {'id':'dosis'}).find_all('li')
    dosis = [d.get_text() for d in dosis]
    
    formas_farmaceuticas = bs.find('div', {'id':'formas'}).find_all('li')
    formas_farmaceuticas = [ff.get_text() for ff in formas_farmaceuticas]
    
    principios_activos = bs.find('div', {'id':"pactivosList"}).find_all('li')
    principios_activos = [pa.get_text() for pa in principios_activos]
    
    excipientes = bs.find('div', {'id':'excipientesList'}).find_all('li')
    excipientes = [e.get_text() for e in excipientes]
    
    caracteristicas = bs.find('div', {'id':'caracteristicasList'}).find_all('li')
    caracteristicas = [c.get_text() for c in caracteristicas]
    
    codigos_atc = bs.find('div', {'id':'atcList'}).find_all('li')
    codigos_atc = [atc.get_text() for atc in codigos_atc]
    
    #Se guardan todos los elementos extraídos anteriormente en una nueva fila cuyo índice en el DataFrame será el número de registro    
    nueva_fila = pd.Series(
        {
            'Número de registro':num_registro ,
            'Medicamento':medicamento,
            'Laboratorio':laboratorio,
            'Autorizado':autorizado_bool,
            'Fecha autorización': autorizado_fecha,
            'Suspendido': suspendido_bool,
            'Fecha suspensión': suspendido_fecha,
            'Comercializado': comercializado_bool,
            'Vías administración': vias_administracion,
            'Dosis': dosis,
            'Formas farmacéuticas': formas_farmaceuticas,
            'Principios activos': principios_activos,
            'Excipientes': excipientes,
            'Características':caracteristicas,
            'Códigos ATC': codigos_atc
        }
        , name=num_registro)  
    return nueva_fila

In [ ]:
tabla =pd.DataFrame([])
for i in meds_ids:  #Recorremos las ids de todos los medicamentos representado a cada uno por i 
    print(i)
    
    #Hacemos click en el elemento de la página web que se identifica por el atributo onclick=i que corresponde al medicamento
    #con el número de registro i  
    driver.find_element(By.CSS_SELECTOR,"div[onclick=\"{}\"]".format(i)).click() 
    
    #Esperamos hasta que se termine de cargar el contenido del html donde se encuentran todos los datos de interés del medicamento
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "figure"))) 
    
    #Accedemos al código fuente de la página una vez que esté se ha terminado de rellenar
    nueva_fila = scrape_data(html=driver.page_source)
    
    #Se añade la nueva fila a la tabla 
    tabla = tabla.append(nueva_fila)
    
    #Se vuelve atrás esperando a que la página cargue para seguir haciendo el mismo proceso para los demás medicamentos
    driver.back()
    driver.implicitly_wait(5)


medicamentoOnSelect('81012')
medicamentoOnSelect('84182')
medicamentoOnSelect('80803')
medicamentoOnSelect('82913')
medicamentoOnSelect('07428001')
medicamentoOnSelect('65382')
medicamentoOnSelect('65059')
medicamentoOnSelect('61836')
medicamentoOnSelect('61837')
medicamentoOnSelect('60014')
medicamentoOnSelect('70452')
medicamentoOnSelect('70451')
medicamentoOnSelect('69223')
medicamentoOnSelect('71437')
medicamentoOnSelect('79696')
medicamentoOnSelect('62389')
medicamentoOnSelect('66460')
medicamentoOnSelect('66459')
medicamentoOnSelect('67763')
medicamentoOnSelect('66458')
medicamentoOnSelect('67621')
medicamentoOnSelect('67988')
medicamentoOnSelect('67874')
medicamentoOnSelect('64920')
medicamentoOnSelect('64921')


In [ ]:
tabla

,Autorizado,Características,Comercializado,Códigos ATC,Dosis,Excipientes,Fecha autorización,Fecha suspensión,Formas farmacéuticas,Laboratorio,Medicamento,Número de registro,Principios activos,Suspendido,Vías administración
81012,1.0,"[USO HOSPITALARIO, Medical prescription]",1.0,"[J05A - ANTIVIRALES DE ACCIÓN DIRECTA, J05AR -...",[600 mg/300 mg],"[CARBOXIMETILALMIDON SODICO TIPO A, AMARILLO A...",26/06/2016,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],Reddy Pharma Iberia S.A.,ABACAVIR/LAMIVUDINA DR. REDDYS 600 MG/300 MG C...,81012,"[ABACAVIR, LAMIVUDINA]",0.0,[VÍA ORAL]
84182,1.0,"[USO HOSPITALARIO, Medical prescription]",1.0,"[J05A - ANTIVIRALES DE ACCIÓN DIRECTA, J05AR -...",[600 mg/ 300 mg],[],30/05/2019,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],"Kern Pharma, S.L.",ABACAVIR/LAMIVUDINA KERN PHARMA 600 MG/300 MG ...,84182,"[ABACAVIR, LAMIVUDINA]",0.0,[VÍA ORAL]
80803,1.0,"[USO HOSPITALARIO, Medical prescription]",1.0,"[J05A - ANTIVIRALES DE ACCIÓN DIRECTA, J05AR -...",[600 mg/300 mg],[],22/09/2016,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],"Sandoz Farmaceutica, S.A.",ABACAVIR/LAMIVUDINA SANDOZ 600 MG/300 MG COMPR...,80803,"[ABACAVIR CLORHIDRATO, LAMIVUDINA]",0.0,[VÍA ORAL]
82913,1.0,"[USO HOSPITALARIO, Medical prescription]",1.0,"[J05A - ANTIVIRALES DE ACCIÓN DIRECTA, J05AR -...",[600 - REVISAR mg],"[CARBOXIMETILALMIDON SODICO TIPO A, AMARILLO A...",30/05/2018,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],"Laboratorio Stada, S.L.",ABACAVIR/LAMIVUDINA STADA 600 MG/300 MG COMPRI...,82913,"[ABACAVIR, LAMIVUDINA]",0.0,[VÍA ORAL]
07428001,1.0,"[USO HOSPITALARIO, Medical prescription, Drivi...",1.0,[L01C - ALCALOIDES DE PLANTAS Y OTROS PRODUCTO...,[100 mg],[],20/01/2009,None,[POLVO PARA DISPERSIÓN PARA PERFUSIÓN],Bristol-Myers Squibb Pharma Eeig,ABRAXANE 5 MG/ML POLVO PARA DISPERSION PARA PE...,07428001,[PACLITAXEL],0.0,[VÍA INTRAVENOSA]
65382,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,"[C10A - AGENTES MODIFICADORES DE LOS LÍPIDOS, ...",[10 mg ezetimiba],"[CROSCARMELOSA SODICA, LACTOSA MONOHIDRATO, LA...",05/05/2003,None,[COMPRIMIDO],"Organon Salud, S.L.",ABSORCOL 10 mg COMPRIMIDOS,65382,[EZETIMIBA],0.0,[VÍA ORAL]
65059,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,[M01A - PRODUCTOS ANTIINFLAMATORIOS Y ANTIRREU...,[8 mg lornoxicam],"[HIDROGENO CARBONATO DE SODIO, PROPILENGLICOL,...",10/11/2002,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],"Grünenthal Pharma, S.A.",ACABEL RAPID 8 mg COMPRIMIDOS RECUBIERTOS CON ...,65059,[LORNOXICAM],0.0,[VÍA ORAL]
61836,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,[M01A - PRODUCTOS ANTIINFLAMATORIOS Y ANTIRREU...,[4 mg lornoxicam],"[CROSCARMELOSA SODICA, LACTOSA]",30/04/1998,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],"Grünenthal Pharma, S.A.",ACABEL 4 mg COMPRIMIDOS RECUBIERTOS CON PELICULA,61836,[LORNOXICAM],0.0,[VÍA ORAL]
61837,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,[M01A - PRODUCTOS ANTIINFLAMATORIOS Y ANTIRREU...,[8 mg],"[CROSCARMELOSA SODICA, LACTOSA MONOHIDRATO]",30/04/1998,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],"Grünenthal Pharma, S.A.",ACABEL 8 mg COMPRIMIDOS RECUBIERTOS CON PELICULA,61837,[LORNOXICAM],0.0,[VÍA ORAL]
60014,1.0,[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA. TRA...,1.0,"[G04B - OTROS PRODUCTOS DE USO UROLÓGICO, INCL...",[1080 mg],[],30/06/1993,None,[COMPRIMIDO DE LIBERACIÓN PROLONGADA],"Ferrer Internacional, S.A.",ACALKA 1080 mg COMPRIMIDOS DE LIBERACION PROLO...,60014,[CITRATO POTASIO],0.0,[VÍA ORAL]


## Versión con scroll

In [6]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)
wait = WebDriverWait(driver, 20)
driver.get('https://cima.aemps.es/cima/publico/home.html')

# Navigate to "listado de búsqueda"
buscador = driver.find_element(By.ID, "inputbuscadorsimple")
buscador.clear()
buscador.send_keys("*")
buscador.send_keys(Keys.ENTER)
sleep(3) # If we do not wait, the `page_source` will not have time to change and it will be showed the previous page (the homepage in this case)
wait.until(EC.presence_of_all_elements_located((By.XPATH, "//*[@id='resultlist']/div"))) 
driver.title


'.:: CIMA ::. Resultados de la búsqueda de medicamentos'

### Diferentes opciones de scrolling

In [ ]:
# Scroll down by a defined number of scrolls
%%time
num_veces_scroll = 5
for i in range(num_veces_scroll): 
    print(f'Found elements so far: {len(driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]"))}')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(3)

Found elements so far: 25
Found elements so far: 50
Found elements so far: 75
Found elements so far: 100
Found elements so far: 125
CPU times: user 85.4 ms, sys: 19.3 ms, total: 105 ms
Wall time: 15.2 s


In [ ]:
# Scroll down by a defined number of elements
# References: https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python
%%time
MAX_ELEMENTS = 125 # Tarda alrededor de 1 hora en obtener 10000 elementos

n_iters = 0
n_meds = 0
while n_meds < MAX_ELEMENTS:
  n_meds =len(driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]"))
  # Scroll hasta el final de la página
  driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  # Esperar hasta que el último elemento de la lista se haya cargado
  wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='resultlist']/div[last()]"))) 
  n_iters += 1

print(f"Se han realizado {n_iters} iteraciones/scrollings para obtener los {n_meds} elementos.")

Se han realizado 176 iteraciones/scrollings para obtener los 125 elementos.
CPU times: user 639 ms, sys: 34.1 ms, total: 673 ms
Wall time: 6.08 s


In [ ]:
# Scroll down until the end of the page
# References: 
#  * https://stackoverflow.com/questions/26566799/wait-until-page-is-loaded-with-selenium-webdriver-for-python
#  * https://www.selenium.dev/selenium/docs/api/py/webdriver_support/selenium.webdriver.support.expected_conditions.html?highlight=expected 
#  * https://sqa.stackexchange.com/questions/38407/how-can-i-use-selenium-to-get-the-last-element-in-a-list
%%time

# Make a diferent search (for reducing the number of elements)
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get('https://cima.aemps.es/cima/publico/home.html')

# Navigate to "listado de búsqueda"
buscador = driver.find_element(By.ID, "inputbuscadorsimple")
buscador.clear()
buscador.send_keys("acetil")
buscador.send_keys(Keys.ENTER)
sleep(3)
wait.until(EC.presence_of_all_elements_located((By.XPATH, "//*[@id='resultlist']/div"))) 

n_iters = 0
while True:
  old_page = driver.page_source
  # Scroll hasta el final de la página
  driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  sleep(2) # NECESITA DEL SLEEP PARA FUNCIONAR CORRECTAMENTE
  # Esperar hasta que el último elemento de la lista se haya cargado
  last_element = driver.find_element(By.XPATH, "//div[@id='resultlist']/div[last()]//div[contains(@onclick,'medicamentoOnSelect')]")
  #last_element = driver.find_element(By.XPATH, "//div[@id='resultlist']/div[last()]").find_element(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]") # Equivalente 
  wait.until(EC.element_to_be_clickable(last_element)) 
  # wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[@id='resultlist']/div[last()]//img"))) # Other option. Not working properly without the sleep either
  print(f"Last element text {last_element.text}")
  n_iters += 1
  current_page = driver.page_source
  if hash(old_page.encode('utf-8')) == hash(current_page.encode('utf-8')):
    break
  n_meds =len(driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]"))
  print(f"Found elements so far: {n_meds}")

print(f"Se han realizado {n_iters} iteraciones/scrollings y se han encontrado un total de {n_meds} elements.")

Last element text ACIDO ACETILSALICILICO CODRAMOL100 MG COMPRIMIDOS GASTRORRESISTENTES EFG
Found elements so far: 50
Last element text DOLVIRAN SUPOSITORIOS
Found elements so far: 75
Last element text TRINOMIA 100MG/20MG/2,5MG CAPSULAS DURAS
Found elements so far: 100
Last element text DENTICELSO SOLUCION
Found elements so far: 107
Last element text DENTICELSO SOLUCION
Se han realizado 5 iteraciones/scrollings y se han encontrado un total de 107 elements.
CPU times: user 200 ms, sys: 56 ms, total: 256 ms
Wall time: 18.9 s


In [ ]:
# Scroll down until the total number of elementss in the search have been found
# References: https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python
%%time

wait = WebDriverWait(driver, 10)
n_iters = 0
n_meds = 0
num_elementos = int(driver.find_element(By.ID, "numResultados").text)
while n_meds < num_elementos:
  n_meds =len(driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]"))
  # Scroll hasta el final de la página
  driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  sleep(0.5)
  # Esperar hasta que el último elemento de la lista sea clicable
  last_element = driver.find_element(By.XPATH, "//div[@id='resultlist']/div[last()]//div[contains(@onclick,'medicamentoOnSelect')]")
  wait.until(EC.element_to_be_clickable(last_element)) 
  # Por motivos informativos, imprimimos cuantas iteraciones/scrollings llevamos
  n_iters += 1
  if n_meds%100 == 0:
    print(f"Already found {n_meds} elements (in {n_iters} iterations)...")

print(f"Se han realizado {n_iters} iteraciones/scrollings para obtener los {n_meds} elementos.")

Already found 100 elements (in 9 iterations)...
Already found 100 elements (in 10 iterations)...
Already found 200 elements (in 20 iterations)...
Already found 200 elements (in 21 iterations)...
Already found 200 elements (in 22 iterations)...
Already found 300 elements (in 29 iterations)...
Already found 300 elements (in 30 iterations)...
Already found 300 elements (in 31 iterations)...
Already found 400 elements (in 41 iterations)...
Already found 400 elements (in 42 iterations)...
Already found 400 elements (in 43 iterations)...
Already found 500 elements (in 53 iterations)...
Already found 500 elements (in 54 iterations)...
Already found 500 elements (in 55 iterations)...
Already found 600 elements (in 64 iterations)...
Already found 600 elements (in 65 iterations)...
Already found 600 elements (in 66 iterations)...
Already found 700 elements (in 76 iterations)...
Already found 700 elements (in 77 iterations)...
Already found 800 elements (in 86 iterations)...
Already found 800 ele

* Con sleep de **1s**: ha tardado 2 horas y 50 min y 641 iteraciones en scrapear los mas de 14000 elementos. En cuanto a recursos, ha consumido 4Gb de RAM.
* Con sleep de **0.5s**: ha tardado 2 horas y 30 mins y 822 iteraciones. Ha consumido casi los mismos recursos que con un 1s.
* Con sleep de **0s**: ha tardado 2 horas y 21 mins y 1553 iteraciones. Ha consumido casi los mismos recursos que con un 1s.

### Scrape de elementos

In [ ]:
%%time
meds = driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]")
meds_ids = [] 
for index,m in enumerate(meds):
    num_registro = m.get_attribute('onclick')
    num_registro = re.search("\d+",num_registro).group(0)
    meds_ids.append(num_registro)
    if index%1000==0:
      print(f"Already treated {index} elementos.")

len(meds_ids)

Already treated 0 elementos.
Already treated 1000 elementos.
Already treated 2000 elementos.
Already treated 3000 elementos.
Already treated 4000 elementos.
Already treated 5000 elementos.
Already treated 6000 elementos.
Already treated 7000 elementos.
Already treated 8000 elementos.
Already treated 9000 elementos.
Already treated 10000 elementos.
Already treated 11000 elementos.
Already treated 12000 elementos.
Already treated 13000 elementos.
Already treated 14000 elementos.
CPU times: user 24.7 s, sys: 1.4 s, total: 26.1 s
Wall time: 8min 57s


In [ ]:
# Guardar identificadores de medicamentos en el Drive
from google.colab import drive
drive.mount('/content/drive')

with open("/content/drive/MyDrive/meds_ids.txt", "w") as out:
  out.write("\n".join(meds_ids))

In [119]:
# Leer identificadores de medicamentos en el Drive
from google.colab import drive
drive.mount('/content/drive')

with open("/content/drive/MyDrive/meds_ids.txt", "r") as f:
    meds_ids = f.read().splitlines()

In [120]:
len(meds_ids)

14334

In [121]:
from selenium.common.exceptions import TimeoutException, NoSuchElementException

driver = webdriver.Chrome('chromedriver',options=chrome_options)
wait = WebDriverWait(driver, 15)

tabla =pd.DataFrame([])
error_meds = {}
for index,med in enumerate(meds_ids):
    try:
      url = 'https://cima.aemps.es/cima/publico/detalle.html?nregistro={}'.format(med)
      driver.get(url)
      
      #Esperamos hasta que se termine de cargar el contenido del html donde se encuentran todos los datos de interés del medicamento
      try:
        #wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "figure"))) # No funciona porque no todos los medicamentos tienen imágenes asociadas
        #wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "#detalleMedicamento > div"))) #No funciona correctamente
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "button[onclick='compartirMedicamento()'"))) #Espera a que cargue el botón de compartir
      except TimeoutException:
        print(f"El medicamento con id {med} ha provocado un TimeoutException. Se procederá a recargar la página y esperar un tiempo por defecto.")
        driver.get(url)
        sleep(5)

      print(f"Iteración nº {index} - Página actual: '{driver.title}'")

      #Accedemos al código fuente de la página una vez que esté se ha terminado de rellenar
      nueva_fila = scrape_data(html=driver.page_source)
      
      #Se añade la nueva fila a la tabla 
      tabla = tabla.append(nueva_fila)

    except Exception as err:
      msg = f"{err} error en la iteración nº {index} - Medicament id: {med} - Página actual: '{driver.title}'.Se guardará esta página para más adelante."
      print(msg)
      error_meds[med] = msg
      continue
    except BaseException:
      driver.quit()
      raise


tabla.shape

Se han truncado las últimas 5000 líneas del flujo de salida.
Iteración nº 9340 - Página actual: 'CIMA: LEVOFLOXACINO MABO-FARMA 500 MG COMPRIMIDOS RECUBIERTOS CON PELICULA EFG'
Iteración nº 9341 - Página actual: 'CIMA: LEVOFLOXACINO MACLEODS 500 MG COMPRIMIDOS RECUBIERTOS CON PELICULA EFG'
Iteración nº 9342 - Página actual: 'CIMA: LEVOFLOXACINO MYLAN 500 mg COMPRIMIDOS RECUBIERTOS CON PELICULA EFG'
Iteración nº 9343 - Página actual: 'CIMA: LEVOFLOXACINO NORMON 5 mg/ml SOLUCION PARA PERFUSION EFG'
Iteración nº 9344 - Página actual: 'CIMA: LEVOFLOXACINO PENSA 500 mg COMPRIMIDOS RECUBIERTOS CON PELICULA EFG'
Iteración nº 9345 - Página actual: 'CIMA: LEVOFLOXACINO PHARMA COMBIX 500 MG COMPRIMIDOS RECUBIERTOS CON PELICULA EFG'
Iteración nº 9346 - Página actual: 'CIMA: LEVOFLOXACINO QUALIGEN 500 mg COMPRIMIDOS RECUBIERTOS CON PELICULA EFG'
Iteración nº 9347 - Página actual: 'CIMA: LEVOFLOXACINO SANOVEL 500 MG COMPRIMIDOS RECUBIERTOS CON PELICULA EFG'
Iteración nº 9348 - Página actual: 'CIMA:

(14329, 15)

* Ha tardado casi 4 horas en scrapear los más de 14000 medicamentos

In [125]:
# Los siguientes medicamentos no se han podido scrapear correctamente
error_meds

{'56683': "'NoneType' object has no attribute 'get_text' error en la iteración nº 421 - Medicament id: 56683 - Página actual: '.:: CIMA ::. Detalle del medicamento...'.Se guardará esta página para más adelante.",
 '57089': "'NoneType' object has no attribute 'get_text' error en la iteración nº 13531 - Medicament id: 57089 - Página actual: '.:: CIMA ::. Detalle del medicamento...'.Se guardará esta página para más adelante.",
 '57720': "'NoneType' object has no attribute 'get_text' error en la iteración nº 714 - Medicament id: 57720 - Página actual: '.:: CIMA ::. Detalle del medicamento...'.Se guardará esta página para más adelante.",
 '80628': "'NoneType' object has no attribute 'get_text' error en la iteración nº 615 - Medicament id: 80628 - Página actual: '.:: CIMA ::. Detalle del medicamento...'.Se guardará esta página para más adelante.",
 '80974': "'NoneType' object has no attribute 'get_text' error en la iteración nº 11210 - Medicament id: 80974 - Página actual: '.:: CIMA ::. Deta

In [124]:
tabla.tail()

,Autorizado,Características,Comercializado,Códigos ATC,Dosis,Excipientes,Fecha autorización,Fecha suspensión,Formas farmacéuticas,Laboratorio,Medicamento,Número de registro,Principios activos,Suspendido,Vías administración
96022009,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,"[N05A - ANTIPSICÓTICOS, N05AH - Diazepinas, ox...",[10 mg],[LACTOSA HIDRATADA],31/10/1996,None,[COMPRIMIDO],Eli Lilly Nederland B.V.,ZYPREXA 10 mg COMPRIMIDOS RECUBIERTOS,96022009,[OLANZAPINA],0.0,[VÍA ORAL]
96022010,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,"[N05A - ANTIPSICÓTICOS, N05AH - Diazepinas, ox...",[10 mg],[LACTOSA HIDRATADA],31/01/1997,None,[COMPRIMIDO],Eli Lilly Nederland B.V.,ZYPREXA 10 mg COMPRIMIDOS RECUBIERTOS,96022010,[OLANZAPINA],0.0,[VÍA ORAL]
96022002,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,"[N05A - ANTIPSICÓTICOS, N05AH - Diazepinas, ox...","[2,5 mg]",[LACTOSA HIDRATADA],31/07/1992,None,[COMPRIMIDO RECUBIERTO],Eli Lilly Nederland B.V.,"ZYPREXA 2,5 mg COMPRIMIDOS RECUBIERTOS",96022002,[OLANZAPINA],0.0,[VÍA ORAL]
96022004,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,"[N05A - ANTIPSICÓTICOS, N05AH - Diazepinas, ox...",[5 mg],[LACTOSA HIDRATADA],31/10/1996,None,[COMPRIMIDO],Eli Lilly Nederland B.V.,ZYPREXA 5 mg COMPRIMIDOS RECUBIERTOS,96022004,[OLANZAPINA],0.0,[VÍA ORAL]
96022006,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,"[N05A - ANTIPSICÓTICOS, N05AH - Diazepinas, ox...","[7,5 mg]",[LACTOSA HIDRATADA],31/10/1996,None,[COMPRIMIDO],Eli Lilly Nederland B.V.,"ZYPREXA 7,5 mg COMPRIMIDOS RECUBIERTOS",96022006,[OLANZAPINA],0.0,[VÍA ORAL]


In [122]:
tabla.to_csv("/content/drive/MyDrive/medicinas_registradas_españa.csv", index=True)

# Problemas encontrados

## Selenium

### StaleElementReferenceException

Selenium no permite iterar sobre un listado de elementos encontrados porque el `id` del elemento cambia cada vez que se regresa la página de resultados de la búsqueda. 

Está explicado en el apartado de "**The Element is not Attached to the DOM"**  en la siguiente página: [https://www.selenium.dev/exceptions/#stale_element_reference](https://www.selenium.dev/exceptions/#stale_element_reference)*texto en cursiva*


In [ ]:
# Search list of medicines
driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]")

[<selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="b1a1df32-a801-4239-bd87-a11ed4d554a5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="35e5bbda-56d3-43ca-8606-aa43302a0a79")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="38599acc-f179-4091-b252-19f5cf3ba306")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="6709c6ce-1dff-40ef-815d-5e4d3d3f1b96")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="d3e0c429-1a47-416b-89b3-31177cc1c65d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="3c1823cb-225a-443b-9238-e40bc368b87a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="3a5a20ff-ca7c-4680-b24b-a0

In [ ]:
# Refresh page and perform the same search
driver.refresh()
driver.implicitly_wait(5)
driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]")

[<selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="dc618bc7-cb7a-448d-ab52-d758b9b076df")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="7dac3da3-ed71-48d3-aab4-4f80aa87ee6e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="6e25dc01-9d39-4ac9-9c02-3bf2c395c818")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="a3e9f86c-f414-4fa6-bf0b-fad52edd5fe5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="51e77872-dc3c-4c47-a094-4c0f6d975030")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="65cc798d-26b4-4046-a233-7228ec44c86a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="9e0c38a2-9846-4113-9b12-ef

Se puede apreciar como realizando la misma búsqueda sobre la misma página, la referencia (identificador del elemento) ha cambiado. Ejemplo: `element="b1a1df32-a801-4239-bd87-a11ed4d554a5"` vs `element="dc618bc7-cb7a-448d-ab52-d758b9b076df"`. Es por esto que al iterar sobre la primera búsqueda y volver atras, no podemos clicar/acceder al siguiente elemento (ya que el id será diferente)

In [ ]:
# Search list of medicines
meds = driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]")
for m in meds:
  m.click() #La excepción salta en el momento de querer acceder al 2º elemento (al 1º sí que puede acceder)
  driver.implicitly_wait(5)
  driver.back()